In [ ]:
#@title Download library and POM
from IPython.display import clear_output

!sudo apt-get install libnetcdf-dev libnetcdff-dev

!mkdir /content/POM
%cd /content/POM

# download grid
for file in ['ETOPO5.dat', 'ETOPO5.head', 'GRID.f', 'IC.dat', 'MATgrid.m', 'MATtopo.m', 'TAU.dat', 'TOPO.dat', 'TOPOhead', 'TS.dat', 'TShead', 'WIND.dat', 'WINDhead', 'bnd.dat', 'dxy.dat', 'gridborder', 'gridcom', 'ijk.dat', 'params', 'plt.dat', 'printout', 'rungrid', 'wnd.dat']:
  !wget http://www.ccpo.odu.edu/POMWEB/GRID-DATA/{file}

# download pom2k
for file in ["pom2k.c","pom2k.f","pom2k.n","runpom2k"]:
  !wget http://www.ccpo.odu.edu/POMWEB/POM2K/{file}

clear_output()

In [ ]:
#@title Hack `pom2k.f` and `pom2k.n`

#@markdown - `pom2k.f`: Uncomment `/*NETCDF*/` code
#@markdown - `pom2k.n`: Fix include `netcdf.inc`

!sed -i '827c\        call write_netcdf(netcdf_file,1)' pom2k.f
!sed -i '828c\        call write_netcdf(netcdf_file,2)' pom2k.f
!sed -i '1456c\            call write_netcdf(netcdf_file,2)' pom2k.f
!sed -i '1476c\              call write_netcdf(netcdf_file,3)' pom2k.f
!sed -i '1519c\        call write_netcdf(netcdf_file,3)' pom2k.f
!sed -i "6014c\        include 'pom2k.n'" pom2k.f
!sed -i "153c\        include '/usr/include/netcdf.inc'" pom2k.n
!sed -i "213c\  " pom2k.n

In [ ]:
#@title 编译并运行POM2k（输出NETCDF文件）
#@markdown # ~~<font color=red> TODO: Support iproblem3 (IC from file) ~~
#@markdown # F**k Fortran, 不修了</font>
#@markdown #### 输入说明
#@markdown |iproblem|problem type|initialisation subroutine|
#@markdown |--------|-------------|--------------------------|
#@markdown |1    |    seamount      | seamount|
#@markdown |2    |    conservation box  |  box |
#@markdown |3    |    IC from file   |file2ic  |

#@markdown - `days` : Run duration in days
#@markdown - `prtd1` : Initial print interval (days)
#@markdown - `dte` : External (2-D) time step (secs.) according to CFL

#@markdown #### 网格大小设置
im = 65 # @param {type:"integer"}
jm = 49 # @param {type:"integer"}
kb = 22 # @param {type:"integer"}

#@markdown #### 运行参数设置
iproblem = "1" # @param ["1", "2", "3"]
days = 0.125 # @param {type:"number"}
prtd1 = 0.0125 # @param {type:"number"}
dte = 6.    # @param {type:"number"}

#@markdown #### NETCDF 输出路径
output_file = "/content/pom2k.nc" # @param {type:"string"}

%cd /content/POM

!echo '      parameter(im={im}, jm={jm}, kb={kb})' >  grid
!echo '      iproblem=  {iproblem}                  ' >  params
!echo '      days=      {days}              ' >> params
!echo '      prtd1=     {prtd1}             ' >> params
!echo '      dte=       {dte}                 ' >> params
!gfortran -O3 -fallow-argument-mismatch pom2k.f -lnetcdff

# output
!./a.out > ./pom2k.nc
!cp -f ./pom2k.nc {output_file}
!rm -f ./pom2k.nc

/content/POM
pom2k.n:548:59:

  537 |         status=nf_put_vara_real(ncid,time_varid,start,count,time)
      |                                                            2
......
  548 |         status=nf_put_vara_real(ncid,uab_varid,start,count,uab)
      |                                                           1
pom2k.n:552:59:

  537 |         status=nf_put_vara_real(ncid,time_varid,start,count,time)
      |                                                            2
......
  552 |         status=nf_put_vara_real(ncid,vab_varid,start,count,vab)
      |                                                           1
pom2k.n:556:59:

  537 |         status=nf_put_vara_real(ncid,time_varid,start,count,time)
      |                                                            2
......
  556 |         status=nf_put_vara_real(ncid,elb_varid,start,count,elb)
      |                                                           1
pom2k.n:568:57:

  537 |         status=nf_put_vara_real(ncid,time